In [6]:
from src.models.decisiontrees.xgboost import XGB1
from src.models.decisiontrees.catboost import CatBoost1
from src.models.decisiontrees.lightgbm import LGBM1
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
historical_metering = pd.read_csv("data/datasets2025/historical_metering_data_ES.csv")
rollout = pd.read_csv("data/datasets2025/rollout_data_ES.csv")

historical_copy = historical_metering.copy()
historical_copy['DATETIME'] = pd.to_datetime(historical_copy['DATETIME'])
historical_copy.set_index('DATETIME', inplace=True)
rollout_copy = rollout.copy()

In [8]:
historical_datetimes = historical_metering["DATETIME"]
rollout_datetimes = rollout["DATETIME"]
common_dates = set(historical_datetimes).intersection(rollout_datetimes)


In [9]:
historical_data = historical_metering[historical_metering["DATETIME"].isin(common_dates)].to_numpy()
rollout_data = rollout[rollout["DATETIME"].isin(common_dates)].to_numpy()
common_dates = list(common_dates)

In [10]:
# fig = plt.figure(figsize=(100, 20))
# plt.plot(common_dates, historical_data[:, 3], label="Historical Data")
# plt.plot(common_dates, rollout_data[:, 3], label="Rollout Data")
# plt.title("Historical and Rollout Data Comparison")
# plt.legend()
# plt.show()

In [11]:
# fig = plt.figure(figsize=(100, 20))
# plt.plot(common_dates, historical_data[:, 3]-rollout_data[:, 3], label="Error")
# plt.title("Historical and Rollout Data Comparison")
# plt.legend()
# plt.show()

In [ ]:

# max_lag = len(historical_copy)
# lags = range(1,max_lag + 1)
# autocorrelations = [historical_copy['VALUEMWHMETERINGDATA_customerES_5'].autocorr(lag=lag) for lag in lags]
# top_10_lags = sorted(range(len(autocorrelations)), key=lambda i: abs(autocorrelations[i]), reverse=True)[:10]
# print("Top 10 Lags:", top_10_lags)

Top 10 Lags: [22626, 0, 1, 167, 2, 166, 335, 168, 334, 336]


/Users/carloteufel/Documents/Projects/Datathon2025/ETH_Datathon_2025/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/carloteufel/Documents/Projects/Datathon2025/ETH_Datathon_2025/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/carloteufel/Documents/Projects/Datathon2025/ETH_Datathon_2025/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [ ]:
cutoff = pd.to_datetime('2024-07-01 00:00:00')

n_customer = 5
customer_data = historical_copy[f'VALUEMWHMETERINGDATA_customerES_{n_customer}']
customer_data = customer_data.to_frame()
customer_data['week_ahead'] = customer_data.shift(-24*7)
training_data = customer_data[customer_data.index < cutoff]
test_data = customer_data[customer_data.index >= cutoff]

In [ ]:
td_1 = training_data[f'VALUEMWHMETERINGDATA_customerES_{n_customer}'].to_numpy()
tt_1 = training_data['week_ahead'].to_numpy()

ted_1 = test_data[f'VALUEMWHMETERINGDATA_customerES_{n_customer}'].to_numpy()
tet_1 = test_data['week_ahead'].to_numpy()

In [ ]:
len_week = 7 * 24 *2
n_weeks = len(training_data) // len_week
X_train = np.empty((n_weeks, len_week))
y_train = np.empty((n_weeks,))

X_test = np.empty((len_week-1, len_week))
y_test = np.empty((len_week-1,))

for i in range(n_weeks-1):
    X_train[i] = td_1[i * len_week:(i + 1) * len_week]
    y_train[i] = tt_1[i * len_week + len_week - 1]

for i in range(1,len_week-1):
    X_test[i] = td_1[-(len_week+i):-i]
    y_test[i] = tt_1[-len_week+i+len_week-1]


In [ ]:
model  = XGB1()
model1  = CatBoost1()
# model2  = LGBM1()

In [ ]:
model.fit(X_train, y_train)
model1.fit(X_train, y_train)
# model2.fit(X_train, y_train)

Learning rate set to 0.026584
0:	learn: 0.0064795	total: 5.02ms	remaining: 5.02s
1:	learn: 0.0064101	total: 7.01ms	remaining: 3.5s
2:	learn: 0.0063540	total: 9.14ms	remaining: 3.04s
3:	learn: 0.0062865	total: 11.3ms	remaining: 2.83s
4:	learn: 0.0062138	total: 13.8ms	remaining: 2.75s
5:	learn: 0.0061441	total: 16ms	remaining: 2.65s
6:	learn: 0.0060869	total: 19ms	remaining: 2.69s
7:	learn: 0.0060168	total: 21.2ms	remaining: 2.63s
8:	learn: 0.0059562	total: 23.3ms	remaining: 2.56s
9:	learn: 0.0058919	total: 25.4ms	remaining: 2.51s
10:	learn: 0.0058288	total: 27.4ms	remaining: 2.47s
11:	learn: 0.0057678	total: 29.6ms	remaining: 2.44s
12:	learn: 0.0057162	total: 31.8ms	remaining: 2.42s
13:	learn: 0.0056602	total: 33.9ms	remaining: 2.39s
14:	learn: 0.0056042	total: 36ms	remaining: 2.36s
15:	learn: 0.0055538	total: 37.9ms	remaining: 2.33s
16:	learn: 0.0055061	total: 40ms	remaining: 2.31s
17:	learn: 0.0054482	total: 42.1ms	remaining: 2.29s
18:	learn: 0.0053979	total: 44.1ms	remaining: 2.28s
1

In [27]:
preds1 = model.predict(X_test)
preds2 = model1.predict(X_test)

preds_avg = (preds1 + preds2) / 2

In [28]:
from src.visualisation.visualisation import visualisation

fig = visualisation(y_pred = preds_avg, y_test=y_test)

In [29]:
fig